<a href="https://colab.research.google.com/github/chavanamol4402/Assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 import speech_recognition as sr

In [2]:
def transcribe_audio(audio_file):
   
    r = sr.Recognizer()

    
    with sr.AudioFile(audio_file) as source:
                
        audio = r.record(source)

    try:
        text = r.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Could not understand the audio.")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")


In [3]:
audio_file_path = "/content/sales_call_telephone_marketers.wav"

In [4]:
trans_text = transcribe_audio(audio_file_path)

In [5]:
print(trans_text)

hello hi Nancy this is Mike from AT&T incorporation yes how can I help you Nancy you have been using our prepaid connection for a couple of years now right yeah that's right how would you like a connection that allows you to make free unlimited voice calls to three AT&T numbers I would love that but what's the catch there's no catch there will be a month


In [18]:
import spacy
import random
from spacy.training import Example

# Prepare your training data
TRAINING_DATA = [
    ("My name is Jeff and I am calling from Amazon.", {"entities": [(11, 15, "caller_name"), (34, 40, "ORG")]}),
    ("I am calling from Microsoft and my name is Satya.", {"entities": [(17, 26, "ORG"), (42, 47, "caller_name")]}),
    ("I am Sundar and this is a call from Google.", {"entities": [(5, 11, "caller_name"), (36, 42, "ORG")]}),
    ("I am calling about your Microsoft Azure subscription.", {"entities": [(13, 31, "caller_name"), (37, 55, "PRODUCT")]}),
    ("This is a call regarding your Google Cloud Platform account.", {"entities": [(21, 44, "caller_name"), (54, 74, "PRODUCT")]}),
    ("I am calling about your Google Cloud Platform account.", {"entities": [(13, 31, "caller_name"), (37, 57, "PRODUCT")]}),
    ("I am Rajesh and this is a call from AT&T.", {"entities": [(5, 11, "caller_name"), (36, 40, "ORG")]}),
    ("My name is John and I am calling from Sansui.", {"entities": [(11, 15, "caller_name"), (34, 40, "ORG")]}),
    ("I am calling from Microsoft and my name is Vishal.", {"entities": [(17, 26, "ORG"), (42, 47, "caller_name")]}),
    ("I am Daniel and this is a call from Jaguar.", {"entities": [(5, 11, "caller_name"), (31, 37, "ORG")]}),
    ("I am calling about your Microsoft Azure subscription.", {"entities": [(13, 31, "caller_name"), (37, 55, "PRODUCT")]}),
    ("This is a call regarding your Google Cloud Platform account.", {"entities": [(21, 44, "caller_name"), (54, 74, "PRODUCT")]}),
    ("I am calling about your Google Cloud Platform account.", {"entities": [(13, 31, "caller_name"), (37, 57, "PRODUCT")]}),
    ("I am Mahesh and this is a call from AT&T.", {"entities": [(5, 11, "caller_name"), (36, 40, "ORG")]}),
]



In [7]:

LABELS = ["caller_name", "ORG", "PRODUCT"]


In [19]:
def train_ner_model(training_data, labels, iterations=30):
    # Load the base model
    nlp = spacy.load("en_core_web_sm")

    # Create the NER component if it doesn't exist
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")

    # Add the entity labels to the NER model
    for label in labels:
        ner.add_label(label)

    # Disable other pipeline components
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        # Initialize the optimizer
        optimizer = nlp.initialize()

        # Train the NER model
        for _ in range(iterations):
            random.shuffle(training_data)
            losses = {}
            for text, annotations in training_data:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)

    # Save the trained NER model
    output_dir = "/content/trained_ner_model"  # Change this path as needed
    nlp.to_disk(output_dir)


In [28]:
# Train the NER model
train_ner_model(TRAINING_DATA, LABELS)

In [29]:
nlp = spacy.load("trained_ner_model")

In [30]:
doc = nlp(trans_text)

In [31]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Nancy ORG
Nancy caller_name
's caller_name
